In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import webbrowser

%matplotlib inline

In [2]:
granola = pd.read_csv('granola_bars.csv')
#protein = pd.read_csv('protein_bars.csv')
#popular = pd.read_csv('popular_bars.csv')

C:\Users\bpawl\AppData\Local\Temp\ipykernel_3208\3183429081.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  granola = pd.read_csv('granola_bars.csv')


In [3]:
granola['Cntstr'].unique()

array([ 1.,  4.,  6., 10.,  5., 35., 12., 20.,  7.,  8., 18., 24., 48.,
       60., 15., 50., nan, 14., 21., 30., 28., 34., 36., 40.])

In [4]:
granola['Cntstr'] = granola['Cntstr'].fillna(0)
granola['Cntstr'] = granola['Cntstr'].astype(int)
granola['Cntstr'] = granola['Cntstr'].astype(str)

In [5]:
granola

,12 Months selected,brand & size,Cntstr,Fisc Pd,Fisc Pd_int,Fisc Yr,Kel Brand,Kel Category,Kel Corporate,Kel Count,...,UnitsNpx,Yago Dol,Yago Dol Ap,Yago Dol Np,Yago Lbs,Yago Lbs Ap,Yago Lbs Np,Yago Units,Yago Units Ap,Yago Units Np
0,False,KNOVA KSH NOT APPLICABLE,1,11,11,2020,KNOVA KSH,PWS,K CRP,01 CT,...,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,False,KNOVA KSH NOT APPLICABLE,1,11,11,2020,KNOVA KSH,PWS,K CRP,01 CT,...,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,False,KNOVA KSH NOT APPLICABLE,4,11,11,2022,KNOVA KSH,PWS,K CRP,04 CT,...,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,False,KNOVA KSH NOT APPLICABLE,6,11,11,2022,KNOVA KSH,PWS,K CRP,06 CT,...,NaN,15418.80,1681.95,13736.85,1677.60,238.47,1439.13,3728.00,529.93,3198.07
4,False,KNOVA KSH NOT APPLICABLE,10,11,11,2022,KNOVA KSH,PWS,K CRP,10 CT,...,NaN,986.57,0.00,986.57,84.44,0.00,84.44,193.00,0.00,193.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69328,False,NTRS PTH NOT APPLICABLE,6,2,2,2023,NTRS PTH,PWS,NTRS PTH CRP,06 CT,...,NaN,3658.83,311.53,3347.30,350.84,31.07,319.77,884.00,78.29,805.71
69329,False,NTRS PTH NOT APPLICABLE,6,3,3,2023,NTRS PTH,PWS,NTRS PTH CRP,06 CT,...,NaN,2645.12,752.69,1892.43,254.00,69.93,184.07,640.00,176.21,463.79
69330,False,NTRS PTH NOT APPLICABLE,6,4,4,2023,NTRS PTH,PWS,NTRS PTH CRP,06 CT,...,NaN,371.07,83.79,287.28,36.51,8.33,28.18,92.00,21.00,71.00
69331,False,NTRS PTH NOT APPLICABLE,6,5,5,2023,NTRS PTH,PWS,NTRS PTH CRP,06 CT,...,NaN,103.74,0.00,103.74,10.32,0.00,10.32,26.00,0.00,26.00


In [46]:
# Initialize Dash app
app = dash.Dash(__name__)


#Create subplots

def create_subplot1(filt_df, x_axis):
    sub1 = filt_df.groupby('Kel Ppg')[x_axis].sum().reset_index()

    fig = go.Figure(data = [go.Bar(x = sub1[x_axis], y = sub1['Kel Ppg'], orientation = 'h')])
    fig.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white', 'font': {'color': 'black'}}, title = f'{x_axis} vs Kel Ppg', xaxis_title = f'{x_axis}', yaxis_title = 'Kel Ppg', title_x = 0.5)
    
    return fig


def create_subplot2(filt_df, x_axis):
    sub2 = filt_df.groupby('Vendor/Brand')[x_axis].sum().reset_index()
    
    fig = go.Figure(data = [go.Bar(x = sub2[x_axis], y = sub2['Vendor/Brand'], orientation = 'h')])    
    fig.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white', 'font': {'color': 'black'}}, title = f'{x_axis} vs Vendor/Brand', xaxis_title = f'{x_axis}', yaxis_title = 'Vendor/Brand', title_x = 0.5)
    
    return fig


def create_subplot3(filt_df, x_axis):
    sub3 = filt_df.groupby('Market/Retailer')[x_axis].sum().reset_index()
    
    fig = go.Figure(data = [go.Bar(x = sub3[x_axis], y = sub3['Market/Retailer'], orientation = 'h')])
    fig.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white', 'font': {'color': 'black'}}, title = f'{x_axis} vs Market/Retailer', xaxis_title = f'{x_axis}', yaxis_title = 'Market/Retailer', title_x = 0.5)
    
    return fig


def create_subplot4(filt_df, x_axis):
    sub4 = filt_df.groupby('Sales Chnl')[x_axis].sum().reset_index()
    
    fig = go.Figure(data = [go.Bar(x = sub4[x_axis], y = sub4['Sales Chnl'], orientation = 'h')])
    fig.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white', 'font': {'color': 'black'}}, title = f'{x_axis} vs Sales Channel', xaxis_title = f'{x_axis}', yaxis_title = 'Sales Channel', title_x = 0.5)
    
    return fig


def create_subplot5(filt_df, x_axis):
    sub5 = filt_df.groupby('Cntstr')[x_axis].sum().reset_index()
    
    fig = go.Figure(data = [go.Bar(x = sub5[x_axis], y = sub5['Cntstr'], orientation = 'h')])
    fig.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white', 'font': {'color': 'black'}}, title = f'{x_axis} vs Count', xaxis_title = f'{x_axis}', yaxis_title = 'Count', title_x = 0.5)
    
    return fig

def create_subplot6(filt_df, x_axis):
    sub6 = filt_df.groupby(['Kel Ppg', 'Fisc Yr'])[x_axis].sum().reset_index()
    
    fig = go.Figure(data = [go.Bar(x = sub6['Fisc Yr'], y = sub6[x_axis])])
    fig.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white', 'font': {'color': 'black'}}, title = f'{x_axis} vs Fisc Yr', xaxis_title = 'Fisc Year', yaxis_title = f'{x_axis}', title_x = 0.5)
    
    return fig


# Define app layout
app.layout = html.Div(style={'backgroundColor': 'white'},
    children = [
    
    # Title
    html.Div([
        html.H1("Granola Bars", style={'text-align': 'center', 'color': 'black', 'font-size': '30px'})
    ]),
    
    # Dropdown fisc yr
    html.Div([
        html.Label('Fisc Yr:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='Fisc Yr',
            options=[{'label': yr, 'value': yr} for yr in ['All'] + granola['Fisc Yr'].unique().tolist()],
            clearable=True,
            multi=True,
            value='All',
            placeholder="Select Fisc Years"
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    # Dropdown vendor brand
    html.Div([
        html.Label('Vendor/Brand:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='Vendor/Brand',
            options=[{'label': brd, 'value': brd} for brd in ['All'] + granola['Vendor/Brand'].unique().tolist()],
            clearable=True,
            multi=True,
            value='All',
            placeholder="Select Vendor/Brand"
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    # Dropdown kel ppg
    html.Div([
        html.Label('Kel Ppg:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='Kel Ppg',
            options=[{'label': ppg, 'value': ppg} for ppg in ['All'] + granola['Kel Ppg'].unique().tolist()],
            clearable=True,
            multi=True,
            value='All',
            placeholder="Select Kel Ppg"
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    # Dropdown sales channel
    html.Div([
        html.Label('Sales Channel:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='Sales Channel',
            options=[{'label': chnl, 'value': chnl} for chnl in ['All'] + granola['Sales Chnl'].unique().tolist()],
            clearable=True,
            multi=True,
            value='All',
            placeholder="Select Sales Channel"
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    # Dropdown market retailer
    html.Div([
        html.Label('Market/Retailer:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='Market/Retailer',
            options=[{'label': mkt, 'value': mkt} for mkt in ['All'] + granola['Market/Retailer'].unique().tolist()],
            clearable=True,
            multi=True,
            value='All',
            placeholder="Select Market/Retailer"
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    # Dropdown count str
    html.Div([
        html.Label('Count:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='Countstr',
            options=[{'label': ct, 'value': ct} for ct in ['All'] + granola['Cntstr'].unique().tolist()],
            clearable=True,
            multi=True,
            value='All',
            placeholder="Select Count"
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    #X AXIS CONTROL
    html.Div([
        html.Label('X-Axis:', style={'color': 'black', 'font-size': '20px', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='X-Axis',
            options=[
                {'label': 'DOL', 'value': 'DOL'},
                {'label': 'Units', 'value': 'Units'}
            ],
            value='DOL', 
            clearable=False
        )
    ], style={'text-align': 'left', 'margin-left': '20px', 'margin-top': '20px'}),
    
    
    # Subplots
    html.Div([
        dcc.Graph(id='subplot1'),
        dcc.Graph(id='subplot2')
    ], style={'display': 'flex', 'flex-direction': 'row'}),  # Display subplots in 2 columns

    html.Div([
        dcc.Graph(id='subplot3'),
        dcc.Graph(id='subplot4')
    ], style={'display': 'flex', 'flex-direction': 'row'}),  # Display subplots in 2 columns

    html.Div([
        dcc.Graph(id='subplot5'),
        dcc.Graph(id='subplot6')
    ], style={'display': 'flex', 'flex-direction': 'row'}),  # Display subplots in 2 columns
    
]
)


# Callback to update all subplots based on slider value and dropdown selections
@app.callback(
     Output('subplot1', 'figure'),
     Output('subplot2', 'figure'),
     Output('subplot3', 'figure'),
     Output('subplot4', 'figure'),
     Output('subplot5', 'figure'),
     Output('subplot6', 'figure'),
     Input('Fisc Yr', 'value'),
     Input('Vendor/Brand', 'value'),
     Input('Kel Ppg', 'value'),
     Input('Sales Channel', 'value'),
     Input('Market/Retailer', 'value'),
     Input('Countstr', 'value'),
     Input('X-Axis', 'value')
)


#Subplot updater
def update_subplots(fiscyr, vendor, kelppg, saleschnl, mktrtl, cnt, x_axis):
    # Filter data based on dropdown selections
    filtered_df = granola  # Assuming granola is your original dataframe
    
    if fiscyr == 'All':
        filtered_df = filtered_df
    elif (not isinstance(fiscyr, list)) & (fiscyr != 'All'):
        filtered_df = filtered_df[filtered_df['Fisc Yr'] == fiscyr]
    elif fiscyr != 'All':
        filtered_df = filtered_df[filtered_df['Fisc Yr'].isin(fiscyr)]
       
    if vendor == 'All':
        filtered_df = filtered_df
    elif (not isinstance(vendor, list)) & (vendor != 'All'):
        filtered_df = filtered_df[filtered_df['Vendor/Brand'] == vendor]
    elif vendor != 'All':
        filtered_df = filtered_df[filtered_df['Vendor/Brand'].isin(vendor)]
        
    if kelppg == 'All':
        filtered_df = filtered_df
    elif (not isinstance(kelppg, list)) & (kelppg != 'All'):
        filtered_df = filtered_df[filtered_df['Kel Ppg'] == kelppg]
    elif kelppg != 'All':
        filtered_df = filtered_df[filtered_df['Kel Ppg'].isin(kelppg)]
        
    if saleschnl == 'All':
        filtered_df = filtered_df
    elif (not isinstance(saleschnl, list)) & (saleschnl != 'All'):
        filtered_df = filtered_df[filtered_df['Sales Chnl'] == saleschnl]
    elif saleschnl != 'All':
        filtered_df = filtered_df[filtered_df['Sales Chnl'].isin(saleschnl)]
        
    if mktrtl == 'All':
        filtered_df = filtered_df
    elif (not isinstance(mktrtl, list)) & (mktrtl != 'All'):
        filtered_df = filtered_df[filtered_df['Market/Retailer'] == mktrtl]
    elif mktrtl != 'All':
        filtered_df = filtered_df[filtered_df['Market/Retailer'].isin(mktrtl)]
       
    if cnt == 'All':
        filtered_df = filtered_df
    elif (not isinstance(cnt, list)) & (cnt != 'All'):
        filtered_df = filtered_df[filtered_df['Cntstr'] == cnt]
    elif cnt != 'All':
        filtered_df = filtered_df[filtered_df['Cntstr'].isin(cnt)]
        
        
    # Create each subplot using the filtered dataframe
    fig1 = create_subplot1(filtered_df, x_axis)
    fig2 = create_subplot2(filtered_df, x_axis)
    fig3 = create_subplot3(filtered_df, x_axis)
    fig4 = create_subplot4(filtered_df, x_axis)
    fig5 = create_subplot5(filtered_df, x_axis)
    fig6 = create_subplot6(filtered_df, x_axis)

    return fig1, fig2, fig3, fig4, fig5, fig6


#Open in a browser
webbrowser.open_new('http://127.0.0.1:8050')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)